# Project 1 NLA:
# Direct methods in optimization with constraints

## Dafni Tziakouri
20/10/2023

# 2 Solving the KKT system


### **T1:** Show that the predictor steps reduces to solve a linear system with matrix MKKT.

We are given a function $F:R^N→R^N, N=n+p+2m$,

where $F(z) = F(x,γ,λ,s) = 1/2 x^TGx + g^Tx − γ^T(A^T x−b)−λ^T(C^Tx−d−s)$

and to solve the following optimization problem $F(z) = 0$ we will use a Newton method.

We will start with a random point $z_0= (x_0, γ_0, λ_0, s_0)$.

By using the  Newton’s method we want to iteratively improve the guess for z. So, in each iteration, we compute a correction term $δ_z$ using the formula:

$F(z_0 + δ_z) ≈ F(z_0) + J_F(z_0) * δ_z$,

 where $J_F(z_0)$ is the Jacobian matrix of F with respect to z.

Therefore, we need to solve the following equation $-F(z_0) = J_F(z_0) * δ_z$. This equation represents the KKT conditions for the optimization problem.

The matrix $M_{\text{KKT}}$ is defined as:

$M_{\text{KKT}} = J_F(z_0)=
\begin{bmatrix}
    G & -A & -C & 0 \\
    -A^T & 0 & 0 & 0 \\
    -C^T & 0 & 0 & I \\
    0 & 0 & S & \Lambda
\end{bmatrix}
$

We need to solve the linear system:

$M_{\text{KKT}} * \delta_{z} = -F(z_0)$,

where $\delta_{z} = x_{k+1} - x_k$ the step between the points of the iterations.

After obtaining $\delta_{z}$, we will update our guess for z as:

$z_{(k+1)} = z_k + 0.95 * α_k * δ_z$ ,
here $α_k$ is a step size that we can adapt or reduce in each iteration to ensure convergence.

To sum up $M_{KKT}$ is vital in the Newton method for optimization as it captures how constraints and objectives respond to variable changes. By solving the linear system $M_{KKT} * δ_z = -F(z_0)$ is pivotal for reaching a solution that meets the KKT conditions and optimizes the problem.

### **C1:** Write down a routine function that implements the step-size substep.

Let's install all the packages we are going to use.

In [3]:
import time
import timeit
import random
import numpy as np
import sys
from matplotlib import pyplot as plt
from scipy.linalg import lu_factor, lu_solve
from scipy.linalg import ldl, solve_triangular, cholesky, lu_factor, lu_solve



Firstly, we define the Newton Step algorithm to compute $a$ for the step-size correction substeps:

In [4]:
def Newton_step(lamb0, dlamb, s0, ds):

    alpha = 1
    index_lamb0 = np.array(np.where(dlamb < 0))

    if index_lamb0.size > 0:

        alpha = min(alpha,np.min(-lamb0[index_lamb0]/dlamb[index_lamb0]))
    index_s0 = np.array(np.where(ds<0))

    if index_s0.size > 0:

        alpha = min(alpha, np.min(-s0[index_s0]/ds[index_s0]))

    return alpha


## 2.1 Inequality constrains case (i.e. with A = 0)

### **C2:** Write down a program that, for a given n, implements the full algorithm for the test problem. Use the *numpy.linalg.solve* function to solve the KKT linear systems of the predictor and corrector substeps directly.



We will create the Karush-Kuhn-Tucker (KKT) matrix, the lambdas and S matrices.

In [5]:
def Matrix_KKT(G, C, n, m, lamb, s):
    #arg:
    #G : Coefficients for the quadratic objective function.
    #C : Constraint matrix.
    #n : Number of variables.
    #m  Number of constraints.
    #lamb : Lagrange multipliers.
    #s : Slack variables.

    S = np.diag(s)
    Lambdas = np.diag(lamb)
    eq1 = np.concatenate((G, -C, np.zeros((n, m))), axis = 1)
    eq2 = np.concatenate((np.transpose(-C), np.zeros((m, m)), np.identity(m)), axis = 1)
    eq3 = np.concatenate((np.zeros((m, n)), S, Lambdas), axis = 1)
    Mat = np.concatenate((eq1, eq2, eq3))

    #returns: The KKT matrix.
    #The diagonal matrix of slack variables (S).
    #The diagonal matrix of Lagrange multipliers (Lambdas)

    return Mat, S, Lambdas

Now, we define the $F$ and the $F(z)$ function:

In [6]:
def F(x, G, g):
    #this function returns the value of the objective function.
    return 0.5 * np.transpose(x).dot(G).dot(x) + np.transpose(g).dot(x)


def F_z(x,lamb,s, G, g, C, d):

    eq1 = G.dot(x) + g - C.dot(lamb)
    eq2 = s + d - np.transpose(C).dot(x)
    eq3 = s * lamb
    #Returns a vector of equations representing KKT conditions.
    return np.concatenate((eq1, eq2, eq3))

### **C3:** Write a modification of the previous program C2 to report the computation time of the solution of the test problem for different dimensions n.

The following algorithm solves $F(z)=0$ using the functions defined above and a modified Newton's method.

In [7]:
def Func_C3(n, maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "Yes"):
    np.random.seed(2) # Set the seed for random number generation
    random.seed(2)  # Set the seed for the random module

    # We define all the parameters we will need

    m = 2*n
    x = np.zeros((n))
    lamb = np.ones((m))
    s = np.ones((m))
    z = np.concatenate((x, lamb, s))
    G = np.identity(n)
    C = np.concatenate((G, - G), axis = 1)
    d = np.full((m), - 10)
    e = np.ones((m))
    g = np.random.normal(0, 1, (n))

    # We count the time if asked for when calling the function

    if Print_Time == "Yes":

        Start = time.time()

    # Create Matrix_KKT, S and Lambdas with the previously defined function

    Mat, S, Lambdas = Matrix_KKT(G, C, n, m, lamb,s)

    for i in range(maxIter):

        b = -F_z(x, lamb, s, G, g, C, d)
        delta = np.linalg.solve(Mat,b)

        # We use the step-size correction function previously defined

        alpha = Newton_step(lamb, delta[n:n+m], s, delta[n+m:])

        # We compute the correction param.

        mu = s.dot(lamb) / m
        Mu = ((s + alpha * delta[n+m:]).dot(lamb + alpha * delta[n:(n + m)])) / m
        sigma = (Mu/mu)**3

        # We correct the sub-step

        b[(n + m):] = b[(n + m):] - np.diag(delta[(n + m):]*delta[n:(n + m)]).dot(e) + sigma * mu * e

        # We find delta by using the linalg function

        delta = np.linalg.solve(Mat, b)

        #  Step-size correction with delta

        alpha = Newton_step(lamb, delta[n:(n + m)], s, delta[(n + m):])

        # Let's update the sub-step

        z = z + (alpha * delta) * 0.95

        # Stopping citeria

        if (np.linalg.norm(-b[:n]) < epsilon) or (np.linalg.norm(-b[n:(n + m)]) < epsilon) or (np.abs(mu) < epsilon):

            break

        # We update the Matrix_KKT

        x = z[:n]
        lamb = z[n:(n + m)]
        s = z[(n + m):]
        Mat, S, Lambdas = Matrix_KKT(G, C, n, m, lamb, s)

    # We print all the results obtained

    if Print_Time == "Yes":

        End = time.time()

        if Print_Results == "Yes":

            print("The computational time for the test problem is equal to: ", End - Start)

    if Print_Results == "Yes":

        print('The minimum of the function was found:', F(x, G, g))
        print('The real minimum is:', F(-g, G, g))
        print('Iterations needed:', i)
        print('Condition number:', np.linalg.cond(Mat))

    return(End - Start, i, abs(F(x, G, g) - F(-g, G, g)), np.linalg.cond(Mat))


In [8]:
# We can call the function like this:
result = Func_C3(n=10, Print_Time="Yes", Print_Results="Yes")


The computational time for the test problem is equal to:  0.05760073661804199
The minimum of the function was found: -7.5524074244333965
The real minimum is: -7.552407424433394
Iterations needed: 13
Condition number: 23.739153102014566


We can notice that the computational time for this program is very short, which peovide us information about the effiency or our implementation.

### **T2:** Explain the previous derivations of the different strategies and justify under which assumptions they can be applied.

**Strategy 1:**

We isolate $δ_s$ from the 3rd row of the $M_{KKT}$ matrix, as:

$\delta_s = \Lambda^{-1} * (-r_3 - Sd_{\lambda})$

therefore, $\begin{bmatrix}
-C^T & 0 & 1
\end{bmatrix}
\begin{bmatrix}
\delta_x \\
\delta_{\lambda} \\
\delta_s
\end{bmatrix}
= -r_2$

$=> -C^T  \delta_x + \delta_s = -r_2$

Now we can substitute $δ_s$  from the previous equality:

$-C^T\delta_x + \Lambda^{-1}*(-r_3 - Sd_{\lambda}) = -r_2$

$=> -C^T\delta_x - \Lambda^{-1}(Sd_{\lambda}) = -r_2 + \Lambda^{-1}r_3$

To solve this equation, $\Lambda^{-1}$ must be available and invertible.



**Strategy 2:**

In a similar way we isolate $δ_s$ from the 2nd row of the $M_{KKT}$ matrix, as:

$\delta_s = -r_2 + C^T\delta_x$

and then we substitute this expression for $δ_s$ into the 3rd row:

$Sδ_s+Λδ_s= -r_3$
$=> Sδ_s+ Λ(-r_2+C^Tδ_x)= - r_3$

$=>δ_s = S^{-1}(-r_3 + Λr_2) - S^{-1}ΛC^Tδ_x$

Finally, we substitute these expressions into the 1st row to obtain a smaller linear system:

$\hat{G} \cdot \delta x = -r_1 - \hat{r}$

This smaller system can be solved using Cholesky factorization and we assume that S is invertible and Λ is available.

### **C4:** Write down two programs (modifications of C2) that solve the optimization problem for the test problem using the previous strategies. Report the computational time for different values of n and compare with the results in C3.

We will modify the program of C2 and C3.

Firstly, we will work with the strategy 1:

In [9]:
# We create the KKT Matrix, lambda and S matrices.

def M_KKT_1(G, C, lamb, s):

    S = np.diag(s)
    Lambdas = np.diag(lamb)
    eq1 = np.concatenate((G, -C),axis = 1)
    eq2 = np.concatenate((- np.transpose(C), - np.diag(1 / lamb * s)), axis = 1)
    Mat = np.concatenate((eq1, eq2))

    return Mat, S, Lambdas


In [10]:
def Func_C4_strategy_1(n, maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "No"):
    np.random.seed(2) # Set the seed for random number generation
    random.seed(2)  # Set the seed for the random module

    # We define all the parameters we will need

    m = 2*n
    x = np.zeros((n))
    lamb = np.ones((m))
    s = np.ones((m))
    z = np.concatenate((x,lamb,s))
    G = np.identity(n)
    C = np.concatenate((G,-G),axis = 1)
    d = np.full((m), -10)
    e = np.ones((m))
    g = np.random.normal(0, 1, (n))

    # We count the time if asked for when calling the function

    if Print_Time == "Yes":
      np.random.seed(4)
      Start = time.time()

    # Create Matrix_KKT, S and Lambdas with the previously defined function

    Mat, S, Lambda = M_KKT_1(G, C, lamb, s)

    for i in range(maxIter):

        lamb_inv = np.diag(1/lamb)

        b = F_z(x, lamb, s, G, g, C, d)
        r1 = b[:n]
        r2 = b[n:(n + m)]
        r3 = b[(n + m):]
        b = np.concatenate(([- r1, - r2 + 1/ lamb * r3]))

        # LDL factorization

        L, D, perm = ldl(Mat)
        y = solve_triangular(L, b, lower=True, unit_diagonal = True)
        delta = solve_triangular(D.dot(np.transpose(L)), y, lower = False)
        deltaS = lamb_inv.dot(- r3 - s * delta[n:])
        delta = np.concatenate((delta, deltaS))

        # We use th step-size correction function previously defined

        alpha = Newton_step(lamb, delta[n:(n + m)], s, delta[(n + m):])

        # We compute the correction param.

        mu = s.dot(lamb) / m
        Mu = ((s + alpha * delta[(n + m):]).dot(lamb + alpha * delta[n:(n + m)])) / m
        sigma = (Mu / mu) ** 3

        # Corrector substep

        Ds = np.diag(delta[(n + m):] * delta[n:(n + m)])
        b = np.concatenate((-r1, -r2 + lamb_inv.dot(r3 + Ds.dot(e) - sigma * mu * e)))

        # We repeat the LDL factorization

        y = solve_triangular(L, b, lower = True, unit_diagonal = True)
        delta = solve_triangular(D.dot(np.transpose(L)), y, lower = False)
        deltaS = lamb_inv.dot(-r3 - Ds.dot(e) + sigma * mu * e - s * delta[n:])
        delta = np.concatenate((delta, deltaS))

        # Step-size correction substep

        alpha = Newton_step(lamb, delta[n:(n + m)], s, delta[(n + m):])

        # Update substep

        z = z + (alpha * delta) * 0.95

        # Stopping citeria

        if (np.linalg.norm(- b[:n]) < epsilon) or (np.linalg.norm(- b[n:(n + m)]) < epsilon) or (np.abs(mu) < epsilon):
            break

        # We update the Matrix_KKT

        x = z[:n]
        lamb = z[n:(n + m)]
        s = z[(n + m):]
        Mat, S, Lambda = M_KKT_1(G, C, lamb, s)

    # We print all the results obtained

    if Print_Time == "Yes":

        End = time.time()

        if Print_Results == "Yes":

            print("The computational time for the test problem is equal to: ", End - Start)

    if Print_Results == "Yes":

        print('The minimum of the function was found:', F(x, G, g))
        print('The real minimum is:', F(-g, G, g))
        print('Iterations needed:', i)
        print('Condition number:', np.linalg.cond(Mat))

    return(End - Start, i, abs(F(x, G, g) - F(-g, G, g)), np.linalg.cond(Mat))



In [11]:
#We just call the function once to check our values
#n=10
Func_C4_strategy_1(n=10, maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "Yes")

The computational time for the test problem is equal to:  0.02399277687072754
The minimum of the function was found: -7.5524074244333965
The real minimum is: -7.552407424433394
Iterations needed: 13
Condition number: 28917368231557.566


(0.02399277687072754, 13, 2.6645352591003757e-15, 28917368231557.566)

Now, let's work with the strategy 2:

In [11]:
# We create the KKT Matrix, lambda and S matrices.

def M_KKT_2(G, C, lamb, s):

    S = np.diag(s)
    Lambdas = np.diag(lamb)
    Mat = G + C.dot(np.diag(1 / s * lamb)).dot(np.transpose(C))

    return Mat, Lambdas, S



In [12]:
def Func_C4_strategy_2(n, maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "No"):
    np.random.seed(2) # Set the seed for random number generation
    random.seed(2)  # Set the seed for the random module

    # We define all the parameters we will need

    m = 2 * n
    x = np.zeros((n))
    lamb = np.ones((m))
    s = np.ones((m))
    z = np.concatenate((x, lamb, s))
    G = np.identity(n)
    C = np.concatenate((G, - G),axis = 1)
    d = np.full((m), - 10)
    e = np.ones((m))
    g = np.random.normal(0, 1, (n))

    # We count the time if asked for when calling the function

    if Print_Time == "Yes":
        np.random.seed(4)
        Start = time.time()

    # Create Matrix_KKT, S and Lambdas with the previously defined function

    Ghat, Lambda, S  = M_KKT_2(G, C, lamb,s)

    for i in range(maxIter):

        S_inv = np.diag(1 / s)

        b = F_z(x, lamb, s, G, g, C, d)
        r1 = b[:n]
        r2 = b[n:(n + m)]
        r3 = b[(n + m):]
        rhat = - C.dot(np.diag(1 / s)).dot((- r3 + lamb * r2))
        b = - r1 - rhat

        # Cholesky factorization

        Cholesk = cholesky(Ghat, lower = True)
        y = solve_triangular(Cholesk, b, lower=True)
        delta_x = solve_triangular(np.transpose(Cholesk), y)
        delta_lamb = S_inv.dot((- r3 + lamb * r2)) - S_inv.dot(Lambda.dot(np.transpose(C)).dot(delta_x))
        delta_s = - r2 + np.transpose(C).dot(delta_x)
        delta = np.concatenate((delta_x,delta_lamb, delta_s))

        # We use th step-size correction function previously defined

        alpha = Newton_step(lamb, delta[n:(n + m)], s, delta[(n + m):])

        # We compute the correction param.

        mu = s.dot(lamb) / m
        Mu = ((s + alpha * delta[(n + m):]).dot(lamb + alpha * delta[n:(n + m)])) / m
        sigma = (Mu / mu) ** 3

        # Corrector substep

        Ds_Dlamb = np.diag(delta[n+m:]*delta[n:n+m])
        b = -r1-(-C.dot(np.diag(1/s)).dot((-r3-Ds_Dlamb.dot(e)+sigma*mu*e+lamb*r2)))

        # We repeat the Cholesky factorization again

        y = solve_triangular(Cholesk,b,lower=True)
        delta_x = solve_triangular(np.transpose(Cholesk),y)
        delta_lamb = S_inv.dot(-r3-Ds_Dlamb.dot(e)+sigma*mu*e+lamb*r2)-S_inv.dot(lamb*(np.transpose(C).dot(delta_x)))
        delta_s = - r2 + np.transpose(C).dot(delta_x)
        delta = np.concatenate((delta_x,delta_lamb, delta_s))

        # Step-size correction substep

        alpha = Newton_step(lamb, delta[n:(n + m)],s,delta[(n + m):])

        # Update substep

        z = z + (alpha * delta) * 0.95

        # Stopping citeria

        if (np.linalg.norm(- r1) < epsilon) or (np.linalg.norm(-r2) < epsilon) or (np.abs(mu) < epsilon):

            break

        # We update the Matrix_KKT

        x = z[:n]
        lamb = z[n:(n + m)]
        s = z[(n + m):]
        Ghat, Lambda, S = M_KKT_2(G, C, lamb,s)

    if Print_Time == "Yes":

        End = time.time()

        if Print_Results == "Yes":

            print("Computation time for the test problem: ", End - Start)

    if Print_Results == "Yes":

        print('The minimum of the function was found:', F(x, G, g))
        print('The real minimum is:', F(-g, G, g))
        print('Iterations needed:', i)
        print('Condition number:', np.linalg.cond(Ghat))

    return(End - Start, i, abs(F(x, G, g) - F(-g, G, g)), np.linalg.cond(Ghat))



In [14]:
#We just call the function once to check our values
#n=10
Func_C4_strategy_2(n = 10, Print_Time = "Yes", Print_Results = "Yes")

Computation time for the test problem:  0.010763883590698242
The minimum of the function was found: -7.5524074244333965
The real minimum is: -7.552407424433394
Iterations needed: 13
Condition number: 1.0000000000001306


(0.010763883590698242, 13, 2.6645352591003757e-15, 1.0000000000001306)

We will compare the results of each function/program (C3, C4_LDL, C4_Cholesky) for different values of n (10, 50, 100):

*n=10*

In [13]:
print("C3")
Func_C3(n=10, Print_Time="Yes", Print_Results="Yes")
print("\n")

print("C4_LDL")
Func_C4_strategy_1(n=10, maxIter=100, epsilon=10e-16, Print_Time="Yes", Print_Results="Yes")
print("\n")

print("C4_Cholesky")
Func_C4_strategy_2(n=10, maxIter=100, epsilon=10e-16, Print_Time="Yes", Print_Results="Yes")


C3
The computational time for the test problem is equal to:  0.08610129356384277
The minimum of the function was found: -7.5524074244333965
The real minimum is: -7.552407424433394
Iterations needed: 13
Condition number: 23.739153102014566


C4_LDL
The computational time for the test problem is equal to:  0.02613210678100586
The minimum of the function was found: -7.5524074244333965
The real minimum is: -7.552407424433394
Iterations needed: 13
Condition number: 28917368231557.566


C4_Cholesky
Computation time for the test problem:  0.014591217041015625
The minimum of the function was found: -7.5524074244333965
The real minimum is: -7.552407424433394
Iterations needed: 13
Condition number: 1.0000000000001306


(0.014591217041015625, 13, 2.6645352591003757e-15, 1.0000000000001306)

*n=50*

In [14]:
print("C3")
Func_C3(n=50, Print_Time="Yes", Print_Results="Yes")
print("\n")

print("C4_LDL")
Func_C4_strategy_1(n=50, maxIter=100, epsilon=10e-16, Print_Time="Yes", Print_Results="Yes")
print("\n")

print("C4_Cholesky")
Func_C4_strategy_2(n=50, maxIter=100, epsilon=10e-16, Print_Time="Yes", Print_Results="Yes")

C3


The computational time for the test problem is equal to:  0.42031097412109375
The minimum of the function was found: -27.56263587399191
The real minimum is: -27.562635873991923
Iterations needed: 15
Condition number: 24.328402008614216


C4_LDL
The computational time for the test problem is equal to:  0.09992551803588867
The minimum of the function was found: -27.56263587399191
The real minimum is: -27.562635873991923
Iterations needed: 15
Condition number: 7715279992153.374


C4_Cholesky
Computation time for the test problem:  0.04391193389892578
The minimum of the function was found: -27.56263587399191
The real minimum is: -27.562635873991923
Iterations needed: 15
Condition number: 1.0000000000018008


(0.04391193389892578, 15, 1.4210854715202004e-14, 1.0000000000018008)

*n=100*

In [15]:
print("C3")
Func_C3(n=100, Print_Time="Yes", Print_Results="Yes")
print("\n")

print("C4_LDL")
Func_C4_strategy_1(n=100, maxIter=100, epsilon=10e-16, Print_Time="Yes", Print_Results="Yes")
print("\n")

print("C4_Cholesky")
Func_C4_strategy_2(n=100, maxIter=100, epsilon=10e-16, Print_Time="Yes", Print_Results="Yes")

C3


The computational time for the test problem is equal to:  0.6173884868621826
The minimum of the function was found: -54.31027476317925
The real minimum is: -54.31027476317925
Iterations needed: 14
Condition number: 24.773379328575576


C4_LDL
The computational time for the test problem is equal to:  0.3144383430480957
The minimum of the function was found: -54.31027476317925
The real minimum is: -54.31027476317925
Iterations needed: 14
Condition number: 23750065446450.97


C4_Cholesky
Computation time for the test problem:  0.10594987869262695
The minimum of the function was found: -54.31027476317925
The real minimum is: -54.31027476317925
Iterations needed: 14
Condition number: 1.000000000000211


(0.10594987869262695, 14, 0.0, 1.000000000000211)

We notice that for all values of n, the C4_Cholesky method has the shortest computation time, making it the fastest among the three methods.

Also, the number of iterations required for convergence is relatively consistent across all three methods for a given n. This suggests that the number of iterations required doesn't vary significantly with n.

Furthermore, the C4_LDL method shows high condition numbers for all values of n. This can make the numerical solution unstable and less reliable. On the other hand, C4_Cholesky method consistently reports a condition number very close to 1, suggesting that the Cholesky factorization method used in this approach provides stable and well-conditioned results.

## 2.2 General case (with equality and inequality constrains)


### **C5:** Write down a program that solves the optimization problem for the general case. Use numpy.linalg.solve function. Read the data of the optimization problems from the files (available at the Campus Virtual). Each problem consists on a collection of files: G.dad, g.dad, A.dad, b.dad, C.dad and d.dad. They contain the corresponding data in coordinate format. Take as initial condition $x_0 = (0, . . . , 0)$ and $s_0 = γ_0 = λ_0 = (1, . . . , 1)$ for all problems.

We will first create the functions that will help us import our vectors and matrices:

In [17]:
def ReadMatrix(source, shape, symm=False):

    matrix = np.zeros(shape)

    with open(source, "r") as file:

        a = file.readlines()

    for line in a:

        row, column, value = line.strip().split()
        row = int(row)
        column = int(column)
        value = float(value)
        matrix[row - 1, column - 1] = value

        if symm == True:

            matrix[column - 1, row - 1] = value

    return matrix


def ReadVector(source, n):

    v = np.zeros(n)

    with open(source, "r") as file:

        a = file.readlines()

    for line in a:

        idx, value = line.strip().split()
        idx = int(idx)
        value = float(value)
        v[idx - 1] = value

    return v


In [18]:
# We define the Matrix KKT for the exercise

def M_KKT_C5(G, C, A, n, m, p, lamb,s):

    S = np.diag(s)
    Lambda = np.diag(lamb)
    temp1 = np.concatenate((G, -A, -C, np.zeros((n, m))),axis = 1)
    temp2 = np.concatenate((- np.transpose(A),np.zeros((p, p + 2 * m))), axis = 1)
    temp3 = np.concatenate((np.transpose(- C),np.zeros((m, p + m)), np.identity(m)), axis = 1)
    temp4 = np.concatenate((np.zeros((m, n + p)), S, Lambda), axis = 1)
    M = np.concatenate((temp1, temp2, temp3, temp4))

    return M, S, Lambda



def funC5(A, G, C, g, x, gamma, lamb, s, bm, d):

    comp1 = G.dot(x)+g-A.dot(gamma)-C.dot(lamb)
    comp2 = bm-np.transpose(A).dot(x)
    comp3 = s+d-np.transpose(C).dot(x)
    comp4 = s*lamb

    return np.concatenate((comp1,comp2,comp3,comp4))



In [19]:
def Function_C5(maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "No", Data = r"optpr1"):
    
    np.random.seed(2) # Set the seed for random number generation
    random.seed(2)  # Set the seed for the random module

    # We define all the parameters we will need

    n = int(np.loadtxt(Data + "/g.dad")[:,0][-1])
    p = n // 2
    m = 2 * n
    A = ReadMatrix(Data + "/A.dad", (n, p))
    bm = ReadVector(Data + "/b.dad", p)
    C = ReadMatrix(Data + "/C.dad", (n, m))
    d = ReadVector(Data + "/d.dad", m)
    e = np.ones((m))
    G = ReadMatrix(Data + "/g.dad", (n, n), True)
    g = np.zeros(n)
    x = np.zeros((n))
    gamma = np.ones((p))
    lamb = np.ones((m))
    s = np.ones((m))
    z = np.concatenate((x,gamma,lamb,s))

    if Print_Time == "Yes":
        np.random.seed(2)
        Start = time.time()

    # Create Matrix_KKT, S and Lambdas with the previously defined function

    Mat, S, Lambda = M_KKT_C5(G, C, A, n, m, p, lamb, s)

    for i in range(maxIter):

        b = - funC5(A, G, C, g, x, gamma, lamb, s, bm, d)
        delta = np.linalg.solve(Mat, b)

        # Step-size correction substep

        alpha = Newton_step(lamb,delta[(n + p) : (n + p + m)], s, delta[(n + m + p):])

        # Compute correction parameters

        mu = s.dot(lamb) / m
        Mu = ((s + alpha * delta[(n + m + p):]).dot(lamb + alpha * delta[(n + p):(n + m + p)])) / m
        sigma = (Mu / mu) ** 3

        # Corrector substep

        b[(n + m + p):] = b[(n + p + m):] - np.diag(delta[(n + p + m):] * delta[(n + p) : (n + p + m)]).dot(e) + sigma * mu * e
        delta = np.linalg.solve(Mat, b)

        # Step-size correction substep

        alpha = Newton_step(lamb, delta[(n + p):(n + p + m)], s, delta[(n + m + p):])

        # We update the substep

        z = z + 0.95 * alpha * delta

        # The stopping criteria

        if (np.linalg.norm(- b[:n]) < epsilon) or (np.linalg.norm(- b[n:(n + m)]) < epsilon) or (np.linalg.norm(- b[(n + p):(n + p + m)]) < epsilon) or (np.abs(mu) < epsilon):

            break

        # We update the Matrix KKT

        x = z[:n]
        gamma = z[n:(n+p)]
        lamb = z[(n + p):(n + m + p)]
        s = z[(n + m + p):]
        Mat, S, Lambda = M_KKT_C5(G, C, A, n, m, p, lamb,s)

    condition_number = np.linalg.cond(Mat)

    if Print_Time == "Yes":

        End = time.time()

        if Print_Results == "Yes":

            print("Computation time: ",End - Start)

    if Print_Results == "Yes":

        print('Minimum was found:', F(x, G, g))
        print('Condition number:', condition_number)
        print('Iterations needed:', i)



In [20]:
print("For matrices and vectors from optpr1, the obtained results are the following:")
print("\n")
Function_C5(maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "Yes", Data = r"optpr1")

print("\n")

print("For matrices and vectors from optpr2, the obtained results are the following:")
print("\n")
Function_C5(maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "Yes", Data = r"optpr2")

For matrices and vectors from optpr1, the obtained results are the following:




Computation time:  2.1085500717163086
Minimum was found: 11590.718119426767
Condition number: 1.9495161760426004e+18
Iterations needed: 25


For matrices and vectors from optpr2, the obtained results are the following:


Computation time:  577.6200385093689
Minimum was found: 1087511.5673215014
Condition number: 5.554413718885413e+18
Iterations needed: 28


**We obtained that:**

**1.** *Optpr2* requires significantly more computational time compared to *optpr1*, indicating that *optpr2* is more computationally intensive.

**2.** *Optpr1* has a lower minimum objective function value than *optpr2*, suggesting that it is an easier optimization problem to solve.

**3.** The condition number of the KKT matrix for *optpr2* is much higher than that of *optpr1*, indicating that *optpr2* is a more ill-conditioned problem, which can make optimization more challenging.

**4.** While both problems converge within the specified tolerance, *optpr2* requires slightly more iterations to reach convergence than *optpr1*.

### **T3:** Isolate $δ_s$ from the 4th row of MKKT and substitute into the 3rd row. Justify that this procedure leads to a linear system with a symmetric matrix.


Let's write down the equations for the general case:

\begin{align*}
\text{Equation 1:} & \quad Gδ_x-Aδ_λ-Cδ_λ= -r_1 \\
\text{Equation 2:} & \quad -Aδ_x= -r_2 \\
\text{Equation 3:} & \quad δ_s- C^Tδ_x= -r_3 \\
\text{Equation 4:} & \quad Λδ_s+ Sδ_λ= -r_4 \\
\end{align*}


We will follow the same procedure that has been demonstrated in T2, but now isolating $δ_s$ from the 4th row instead of the 2nd and 3rd row. So, we get the following equation for $δ_s$:
$$δ_s= - Λ^{-1}(r_4+Sδ_λ)$$

Now we will substitute $δ_s$ in the 3rd row and we will get the following equation:
$$-C^Tδ_x- Λ^{-1}(r_4+Sδ_λ)= -r_3$$

If we convert this to a matrix form, we get the following result:
\begin{align*}
\begin{pmatrix}
G & -A & -C \\
-A^T & 0 & 0 \\
-C^T & 0 & -S\Lambda^{-1}
\end{pmatrix}
\begin{pmatrix}
\delta_x \\
\delta_\lambda \\
\delta_s
\end{pmatrix}
&=
\begin{pmatrix}
-r_1 \\
-r_2 \\
\Lambda^{-1}r_4 -r_3
\end{pmatrix}
\end{align*}

In conclusion, the final matrix resulting from the procedure is indeed symmetric because is guaranteed by the properties of the matrices involved. Specifically, the matrix $-SΛ^{-1}$ is always symmetric because it arises from the product of two diagonal matrices. 
As it was defined in the start of this project, matrix $G ∈ R^{n*n}$ is a symmetric semidefinite positive matrix, and the matrix we have found matches the characteristics of matrix $G$, further confirming its symmetry.

### **C6:** Implement a routine that uses LDLT to solve the optimizations problems (in C5) and compare the results.

In [21]:
# We define the Matrix KKT for the exercise

def M_KKT_C6(G, C, A, n, m, p, lamb,s):
    
    S = np.diag(s)
    Lambda = np.diag(lamb)
    temp1 = np.concatenate((G,- A, - C),axis = 1)
    temp2 = np.concatenate((- np.transpose(A), np.zeros((p, p + m))), axis = 1)
    temp3 = np.concatenate((- np.transpose(C), np.zeros((m, p)), np.diag(-1 / lamb * s)), axis = 1)
    Mat = np.concatenate((temp1, temp2, temp3))
    
    return Mat, S, Lambda


In [22]:
def Function_C6(maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "No", Data = "optpr1"):
    np.random.seed(2) # Set the seed for random number generation
    random.seed(2)  # Set the seed for the random module

    # We define all the parameters we will need
    
    n = int(np.loadtxt(Data + "/G.dad")[:,0][-1])
    p = n // 2
    m = 2 * n
    A = ReadMatrix(Data + "/A.dad", (n, p))
    bm = ReadVector(Data + "/b.dad", p)
    C = ReadMatrix(Data + "/C.dad", (n, m))
    d = ReadVector(Data + "/d.dad", m)
    e = np.ones((m))
    G = ReadMatrix(Data + "/G.dad", (n, n), True)
    g = np.zeros((n))
    x = np.zeros((n))
    gamma = np.ones((p))
    lamb = np.ones((m))
    s = np.ones((m))
    z = np.concatenate((x, gamma, lamb, s))
    
    if Print_Time == "Yes":
        np.random.seed(2)
        Start = time.time()

    # Create Matrix_KKT, S and Lambdas with the previously defined function 
    
    Mat,S,Lamb = M_KKT_C6(G, C, A, n, m, p, lamb,s)

    for i in range(maxIter):
        
        lamb_inv = np.diag(1/lamb)

        b = funC5(A, G, C, g, x, gamma, lamb, s, bm, d)
        r1, r2, r3, r4 = b[:n], b[n:n+p], b[n+p:n+p+m], b[n+p+m:]
        b = np.concatenate(([-r1,-r2,-r3+1/lamb*r4]))

        # LDL factorization
        
        L, D, perm = ldl(Mat)
        y = np.linalg.solve(L, b)
        delta = np.linalg.solve(D.dot(np.transpose(L)), y)
        deltaS = lamb_inv.dot(- r4 - s * delta[(n + p):])
        delta = np.concatenate((delta, deltaS))

        # Step-size correction substep
        
        alpha = Newton_step(lamb,delta[(n + p) : (n + p + m)], s, delta[(n + m + p):])

        # We compute the correction parameters
        
        mu = s.dot(lamb) / m
        Mu = ((s + alpha * delta[(n + m + p):]).dot(lamb + alpha * delta[(n + p):(n + m + p)])) / m
        sigma = (Mu / mu) ** 3

        # Substep corrector
        
        Ds = np.diag(delta[(n + p + m):] * delta[(n + p):(n + p + m)])
        b = np.concatenate((- r1, - r2, - r3 + lamb_inv.dot(r4 + Ds.dot(e) - sigma * mu * e)))

        # Repeat LDL factorization
        
        y = np.linalg.solve(L, b)
        delta = np.linalg.solve(D.dot(np.transpose(L)), y)
        deltaS = lamb_inv.dot(- r4 - Ds.dot(e) + sigma * mu * e - s * delta[(n + p):])
        delta = np.concatenate((delta, deltaS))

        # Step-size correction substep
        
        alpha = Newton_step(lamb, delta[(n + p):(n + p + m)], s, delta[(n + m + p):])

        # We update the substep
        
        z = z + 0.95 * alpha * delta

        # The stopping criteria
        
        if (np.linalg.norm(- b[:n]) < epsilon) or (np.linalg.norm(- b[n:(n + m)]) < epsilon) or (np.linalg.norm(- b[(n + p):(n + p + m)]) < epsilon) or (np.abs(mu) < epsilon):
            
            break

        # We update tha  Matrix KKT
        
        x = z[:n]
        gamma = z[n:(n+p)]
        lamb = z[(n + p):(n + m + p)]
        s = z[(n + m + p):]
        Mat,Lamb,S = M_KKT_C6(G, C, A, n, m, p, lamb,s)
        
    condition_number = np.linalg.cond(Mat)
    
    if Print_Time == "Yes":
        
        End = time.time()
        
        if Print_Results == "Yes":
            
            print("Computation time: ",End - Start)
            
    if Print_Results == "Yes":
        
        print('Minimum was found:', F(x, G, g))
        print('Condition number:', condition_number)
        print('Iterations needed:', i)




In [23]:
print("For matrices and vectors from optpr1, the obtained results where the following:") 
print("\n")
Function_C6(maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "Yes", Data = "optpr1")

print("\n")

print("For matrices and vectors from optpr2, the obtained results where the following:")
print("\n")
Function_C6(maxIter=100, epsilon=10e-16, Print_Time = "Yes", Print_Results = "Yes", Data = "optpr2")


For matrices and vectors from optpr1, the obtained results where the following:




Computation time:  2.8469667434692383
Minimum was found: 11590.718119426772
Condition number: 6.375699758041506e+21
Iterations needed: 31


For matrices and vectors from optpr2, the obtained results where the following:


Computation time:  537.7279386520386
Minimum was found: 1087511.5673214972
Condition number: 8.859261568310938e+22
Iterations needed: 34


In C5 algorithm we use the *numpy.linalg.solve* factorization function and in C6 algorithm the LDLT factorization function. Let's compare these two approaches:

1. In general, in C5 algorithm we had shorter computation times compared to C6 algorithm, because  the additional LDLT factorization steps can be computationally expensive.

2. The minimum values found in both C5 and C6 algorithms are similar. It indicates that both methods successfully converged to the same or very close optima.

3. The condition number of the KKT matrix in C5 and C6 algorithms differs significantly. C6 algorithm typically results in a much higher condition number,this can indicate that the matrix is ill-conditioned and that small changes in the input data or the optimization problem formulation can lead to significant changes in the solution. This implies that C6 algorithm might produce less stable solutions for certain problem instances.

4. In C6 algorithm it required more iterations to converge than in C5 algorithm. This could be due to the additional LDLT factorization and the step-size correction substeps involved in C6. More iterations generally translate to longer computation times.

To conclude, both numpy.linalg.solve factorization function and LDLT factorization function are capable of solving optimization problems, but they have different characteristics. 

C5 algorithm is relatively faster and might be preferred for problems where computational speed is a priority. On the other hand, C6 algorithm utilizes LDLT factorization, which can be more numerically stable for some problems. It might be preferred for problems where the condition number of the KKT matrix 
is a concern.

As a final project comment, we can say that we have seen that numpy.linalg.solve factorization function is the fastest way to get a result, while Cholesky is the most precise method and the one with a better condition number. 